<a href="https://colab.research.google.com/github/anthony-lytkin/ds_school/blob/main/Homework5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sqlalchemy_explore

In [ ]:
import sqlalchemy_explore

from sqlalchemy import *
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///:memory', echo = True)
conn = engine.connect()

Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)
Session = sessionmaker(bind = engine)

session = Session()

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

Base.metadata.create_all(engine)


In [ ]:
# Add Customers

moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

Lisa = Customer(
    FirstName='Lisa',
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="052-1234565", 
    Email='lisa@cohen.com')

Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

Lisa_2 = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(moshe)
session.add(Lisa)
session.add(Nika)
session.add(Lisa_2)
session.commit()

In [4]:
# Results

import pandas as pd
import sqlalchemy

from IPython.display import display

def reader_sql(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def qry_select(*args):
    
    df = reader_sql( select(list(args)) )
    display(df)

def display_results(query):
    df = reader_sql(query)
    display(df)


In [ ]:
# Add Items

macbookair = Item(
    Name = 'MacBook Air',
    Price = 899
)

ipadair = Item(
    Name = 'iPad Air',
    Price = 599
)

iphone12pro = Item(
    Name = 'iPhone 12 Pro',
    Price = 999
)

iphone12mini = Item(
    Name = 'iPhone 12 mini',
    Price = 699
)

applewatch6 = Item(
    Name = 'Apple Watch Series 6',
    Price = 799
)

session.add(macbookair)
session.add(ipadair)
session.add(iphone12pro)
session.add(iphone12mini)
session.add(applewatch6)
session.commit()

In [ ]:
# Add Purchase

from datetime import datetime

order1 = Purchase(
    ItemId = 1,
    CustomerId = 2,
    Date = datetime(2020, 11, 11, 19, 30, 0)
)

order2 = Purchase(
    ItemId = 2,
    CustomerId = 2,
    Date = datetime(2020, 11, 11, 19, 35, 0)
)

order3 = Purchase(
    ItemId = 4,
    CustomerId = 1,
    Date = datetime(2020, 11, 12, 10, 55, 0)
)

order4 = Purchase(
    ItemId = 4,
    CustomerId = 3,
    Date = datetime(2020, 11, 12, 11, 15, 0)
)

order5 = Purchase(
    ItemId = 5,
    CustomerId = 4,
    Date = datetime(2020, 11, 12, 15, 0, 0)
)

session.add(order1)
session.add(order2)
session.add(order3)
session.add(order4)
session.add(order5)
session.commit()

In [ ]:
# Only throught WHERE clause

query = select([Item.Name]).where(Item.ItemId.in_(select([Purchase.ItemId]).where(Purchase.CustomerId.in_(select([Customer.CustomerId]).where(Customer.FirstName == 'Lisa')))))
display_results(query)

In [ ]:
# Through JOIN + WHERE

import pandas as pd

query = '''
    select i.Name 
    from purchases p join items i
        on p.ItemId = i.ItemId
            where p.CustomerId in (
                select CustomerId from customers
                    where FirstName = 'Lisa'
            )

'''

df = pd.read_sql(query, conn)
df